In [36]:
import rasterio
from rasterio import features
from rasterio.features import shapes
from shapely.geometry import shape
import satellite_images_nso.api.nso_georegion as nso
import geopandas as gpd
from shapely.geometry import Polygon, Point
import numpy as np
import pandas as pd
from shapely.geometry import MultiPoint, MultiLineString

In [37]:
raster_path = "E:/data/coepelduynen/20230910_105008_PNEO-03_1_1_30cm_RD_12bit_RGBNED_NoordwijkAanZee_natura2000_coepelduynen_cropped_ndvi_re_ndvi_asphalt_crop.tif"
georegion_path = "C:/repos/satellite-images-nso/input_data/natura2000_coepelduynen.geojson"

In [38]:
dataset = rasterio.open(raster_path)
data = rasterio.open(raster_path).read()
column_names: list = [
            "r",
            "g",
            "b",
            "n",
            "e",
            "d",
            "ndvi",
            "re_ndvi"
        ]

In [39]:
z_shape = data.shape[0]
x_shape = data.shape[1]
y_shape = data.shape[2]
x_coordinates = [
            [x for y in range(0, data.shape[2])]
            for x in range(0, data.shape[1])
        ]
y_coordinates = [
            [y for y in range(0, data.shape[2])] for x in range(0, data.shape[1])
        ]

rd_x, rd_y = rasterio.transform.xy(
            dataset.transform, x_coordinates, y_coordinates, offset="center"
        )


data = np.append(data, rd_x).reshape([z_shape + 1, x_shape, y_shape])
data = np.append(data, rd_y).reshape([z_shape + 2, x_shape, y_shape])

data = data.reshape(-1, x_shape * y_shape).transpose()

df = pd.DataFrame(
            data,
            columns=column_names
            + [
                "rd_x",
                "rd_y",
            ],
        )

In [40]:
# Filter empty pixels
df = df[df['b'] != 0]

In [41]:
gdf1 = gpd.GeoDataFrame(df[0:120000], geometry=gpd.points_from_xy(df[0:120000]["rd_x"], df[0:120000]["rd_y"]))
gdf1 = gdf1.set_crs('EPSG:28992').to_crs("EPSG:4326")

In [42]:
gdf2 = gpd.read_file(georegion_path) 

In [43]:
intersection = gpd.overlay(gdf1, gdf2, how='difference')

In [44]:
intersection

,r,g,b,n,e,d,ndvi,re_ndvi,rd_x,rd_y,geometry
0,288.0,372.0,481.0,447.0,362.0,653.0,121.0,111.0,89744.55,472287.75,POINT (4.43193 52.23479)


In [11]:
intersection.to_file("missing_region.geojson")